In [26]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Hand Tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8)

# Drawing canvas and variables
canvas = np.zeros((480, 640, 3), dtype=np.uint8)
prev_x, prev_y = None, None
smooth_factor = 0.3  # Increased smoothness

# Colors
colors = {
    "red": (0, 0, 255),
    "blue": (255, 0, 0),
    "green": (0, 255, 0),
    "white": (255, 255, 255)
}
current_color = colors["white"]  # Default color
eraser_mode = False

def smooth_point(new, old):
    """Applies weighted smoothing for a more stable pointer."""
    if old is None:
        return new
    return int(smooth_factor * new + (1 - smooth_factor) * old)

def detect_gesture(landmarks):
    """Identifies user gestures for color selection, writing, and erasing."""
    global current_color, eraser_mode, canvas

    # Get landmark positions
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP].y
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP].x
    thumb_ip = landmarks[mp_hands.HandLandmark.THUMB_IP].x

    # Measure distance between fingers to confirm gestures
    def is_finger_up(finger_tip, other_finger_tip, threshold=0.02):
        return abs(finger_tip - other_finger_tip) > threshold

    # Gesture for Clearing Canvas (Closed Fist)
    if all(abs(landmarks[i].x - landmarks[i + 1].x) < 0.02 for i in range(4, 8)):
        canvas[:] = 0
        print("🗑️ Canvas Cleared!")

    # Gesture for Color Selection
    if is_finger_up(pinky_tip, ring_tip):
        current_color = colors["red"]
        print("🎨 Color Changed: Red")

    elif is_finger_up(ring_tip, middle_tip):
        current_color = colors["blue"]
        print("🎨 Color Changed: Blue")

    elif is_finger_up(middle_tip, index_tip):
        current_color = colors["green"]
        print("🎨 Color Changed: Green")

    # Writing Mode (Index Finger is up)
    if is_finger_up(index_tip, middle_tip):
        print("✏️ Writing Mode")

    # Eraser Mode (Thumb touching index finger)
    if abs(thumb_tip - thumb_ip) < 0.05:
        eraser_mode = True
        print("🧽 Eraser Mode Activated")
    else:
        eraser_mode = False

# Start Video Capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip for natural interaction
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            detect_gesture(hand_landmarks.landmark)  # Detect gestures

            # Get index finger tip position
            index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger.x * 640), int(index_finger.y * 480)

            # Apply smoothing
            x = smooth_point(x, prev_x)
            y = smooth_point(y, prev_y)

            if prev_x is not None and prev_y is not None:
                thickness = 10 if eraser_mode else 3  # Thicker for eraser
                color = (0, 0, 0) if eraser_mode else current_color  # Black for eraser
                
                cv2.line(canvas, (prev_x, prev_y), (x, y), color, thickness)

            prev_x, prev_y = x, y  # Update previous point

    # Overlay drawing on frame
    output = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
    
    # Display the current color on screen
    cv2.rectangle(output, (10, 10), (50, 50), current_color, -1)

    cv2.imshow("Air Notepad", output)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Red
✏️ Writing Mode
🎨 Color Changed: Blue
✏️ Writing Mode
🧽 Eraser Mode Activated
🎨 Color Changed: Blue
✏️ Writing Mode
🎨 Color Changed: Blue
✏️ Writing Mode
🎨 Color Changed: Blue
✏️ Writing Mode
🎨 Color Changed: Blue
✏️ Writing Mode
🎨 Color Changed: Blue
✏️ Writing Mode
🎨 Color Changed: Blue
✏️ Writing Mode
🧽 Eraser Mode Activated
🎨 Color Changed: Blue
✏️ Writing Mode
🧽 Eraser Mode Activated
🎨 Color Changed: Blue
✏️ Writing Mode
🧽 Eraser Mode Activated
🎨 Color Changed: Blue
✏️ Writing Mode
🧽 Eraser Mode Activated
🎨 Color Changed: Blue
✏️ Writing Mode
🧽 Eraser Mode Acti